In [53]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import numpy as np
import datetime

In [54]:
def clean(string):
    """
    Clean the script tag contents for easier retrieval of data.
    
    paramters:
        string: str.
        The contents of the script tag.
        
    returns:
        string: str.
        The cleaned contents of the script tag.
    """
    
    string = re.sub("[\n \\\']",'',str(string))
    string = string.replace(" ",'')
    string = re.sub('[{}\[\]():]',' ',string)
    string = re.sub('[\"\" /*]','',string)
    
    return string

In [55]:
def retrieve_dates(string):
    """
    Retrieve dates from the cleaned script tag contents.
    
    parameters:
        string: str.
        The cleaned contents of the script tag.
        
    returns:
        dates: list.
        A list of the dates.
    """
    
    start_string = 'categories'
    end_string = ',yAxis'
    
    start_index = string.find(start_string) + len(start_string)
    end_index = string.find(end_string)
    
    dates = string[start_index:end_index].strip().split(",")
    
    return dates

In [56]:
def retrieve_daily_stats(string):
    """
    Retrieves daily statistics from the cleaned script tag contents.
    
    parameters:
        string: str.
        Cleaned contents of the script tag.
                
    returns:
        values: list.
        A list of daily statistics ordered by date.
    """
    
    start_string = 'data'
    end_string = ',name'
    
    start_index = string.find(start_string) + len(start_string)
    end_index = string.find(end_string)
    
    values = string[start_index:end_index].strip().split(",")
    
    return values

In [57]:
def retrieve_overall_stats(string):
    """
    Retrieves overall statistics from the cleaned script tag contents.
    
    parameters:
        string: str.
        Cleaned contents of the script tag.
        
    returns:
        values: list.
        A list of daily statistics ordered by date.
    """
    
    start_string = 'data'
    end_string = ',resp'
    
    start_index = string.find(start_string) + len(start_string)
    end_index = string.find(end_string)
    
    values = string[start_index:end_index].strip().split(",")
    
    return values

In [58]:
countries = [
    'us','brazil','russia','spain',
    'italy','france','germany',
    'turkey','india','iran','peru',
    'canada','chile','china','mexico',
    'saudi-arabia','pakistan','belgium',
    'qatar', 'bangladesh',
    'belarus', 'ecuador', 'sweden'
]


data_indexes = {
                'total_cases':0,
                'daily_cases':1,
                'active_cases':2,
                'total_deaths':3,
                'daily_deaths':4
               }

In [59]:
def page_contents(url):
    """
    Retrieves contents of the web page from the specified url and the specific div tag class - col-md-12
    
    paramters:
        url: str.
        The url to the web page to be scrapped.
        
    returns:
        result: str.
        HTML parsed web page content as string.
    """
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    result = soup.find_all('div', class_= 'col-md-12')
    
    return result

In [60]:
def script_tag_contents(page_content, stat):
    """
    Retrieves the script tag contents from the web page contents.
    
    paramters:
        page_content: str.
        HTML parsed web page contents
        
        stat: str.
        String specifying the kind of statistic from the data_indexes.
        
    returns:
        script_content: str.
        Script tag contents as string.
    """
    
    stat_data = page_content[data_indexes[stat]]
    script_content = stat_data.find('script').contents[0]
    
    return script_content

In [61]:
def build_dataframe(values,stat_name,dataframe=None,date=None):
    """
    Build a DataFrame from the dates and the values scraped.
    
    parameters:
        dataframe: DataFrame.
        A DataFrame containing dates and/or statistics.
        
        date: list.
        List of dates for the statistics in string format.
        
        values: list:
        List of values (data) for the statistics in string format.
        
        stat_name: str.
        Name of the statistic, for which the list of values are passed.
        
        returns:
            dataframe: DataFrame.
            DataFrame containing dates and passed statistic values.
    """
    
    if dataframe is None and date is not None:
        
        dataframe = pd.DataFrame({'date':date, stat_name:values})
        
    else:
        
        dataframe[stat_name] = values
        
    return dataframe

In [62]:
def clean_date(dataframe, date_col):
    """
    Clean the date column in the dataframe to standard date representation - YYYY-MM-DD
    
    parameters:
        dataframe: DataFrame.
            DataFrame whose dates are to be cleaned.
            
        date_col: str.
        Name of the date column in the DataFrame.
        
    returns:
        dataframe: DataFrame.
        Cleaned DataFrame.
    """
    
    dataframe[date_col] = dataframe[date_col].apply(
        lambda date: date[:3]+" "+date[3:]+" 2020"
    )
    dataframe[date_col] = dataframe[date_col].apply(
        lambda date: datetime.datetime.strptime(date,'%b %d %Y').date()
    )
    
    return dataframe

In [63]:
def scrape_data():
    """
    Scrape the web page for date, total cases, daily cases, total active cases, total_deaths, daily deaths
    daily recoveries per country. Creates a folder in local directory containing csv files per country with 
    the respective data. Website - worldometers.info
    
    parameters: None
    
    returns: bool.
    boolean.
    """
    
    has_been_run_once = False
        
    for country in countries:
                
        url = "https://www.worldometers.info/coronavirus/country/"+country+"/"
        content = page_contents(url)
        
        for stat in data_indexes:
                        
            script_contents = script_tag_contents(content, stat)
            script_contents = clean(script_contents)
            
            if 'daily' in stat:
                
                data = retrieve_daily_stats(script_contents)
            
            else:
                
                data = retrieve_overall_stats(script_contents)
                
            if not has_been_run_once:
                
                date = retrieve_dates(script_contents)
                dataframe = build_dataframe(data, stat, date= date)
                has_been_run_once = True
                
            else:
                
                dataframe = build_dataframe(data, stat, dataframe= dataframe)
            
        dataframe = clean_date(dataframe, date_col='date')
        dataframe.to_csv('./Data/covid19_'+country+'_stats.csv',index=False)
        has_been_run_once = False
        print("Scraped successfully: ",country)
    
    return True

In [287]:
scrape_data()

Scraped successfully:  us
Scraped successfully:  brazil
Scraped successfully:  russia
Scraped successfully:  spain
Scraped successfully:  italy
Scraped successfully:  france
Scraped successfully:  germany
Scraped successfully:  turkey
Scraped successfully:  india
Scraped successfully:  iran
Scraped successfully:  peru
Scraped successfully:  canada
Scraped successfully:  chile
Scraped successfully:  china
Scraped successfully:  mexico
Scraped successfully:  saudi-arabia
Scraped successfully:  pakistan
Scraped successfully:  belgium
Scraped successfully:  qatar
Scraped successfully:  bangladesh
Scraped successfully:  belarus
Scraped successfully:  ecuador
Scraped successfully:  sweden


True

In [90]:
df = pd.read_csv('./Data/covid19_brazil_stats.csv')
df

,date,total_cases,daily_cases,active_cases,total_deaths,daily_deaths
0,2020-02-15,0,NaN,0,0,NaN
1,2020-02-16,0,NaN,0,0,NaN
2,2020-02-17,0,NaN,0,0,NaN
3,2020-02-18,0,NaN,0,0,NaN
4,2020-02-19,0,NaN,0,0,NaN
...,...,...,...,...,...,...
100,2020-05-25,376669,13051.0,199314,23522,806.0
101,2020-05-26,392360,15691.0,209218,24549,1027.0
102,2020-05-27,414661,22301.0,222317,25697,1148.0
103,2020-05-28,438812,24151.0,218867,26764,1067.0


In [66]:
def table_contents(url):
    """
    Retrieve contents of the table on the url.
    
    parameters:
        url: str.
        URL of the webpage. The table contents are scraped from this.
        
    returns:
        table_data: str.
        HTML parsed table data in string format.
    """
    
    page_content = requests.get(url)
    soup = BeautifulSoup(page_content.content, 'html.parser')
    table_data = soup.find('table', id='thetable')
    
    return table_data

In [217]:
def country_names(table_data):
    """
    Scrape country names from the table contents scraped from the Wikipedia url.
    
    parameters:
        table_data: str.
        Scraped table data from webpage in string format.
        
    returns:
        countries: list.
        A list of country names from the table on webpage.
    """
    
    countries= []
    table_head_data = table_data.find_all('th', scope= 'row')
    
    for data in table_head_data:
        
        anchor_data = data.find('a')
        
        if anchor_data is not None:
            
            if anchor_data.find('i'):
                
                countries.append(anchor_data.i.contents[0])
                continue
            
            countries.append(anchor_data.contents[0])
    
    return countries

In [68]:
def chunks(lst, size):
    """
    Segments the input list into equal chunks of specified size.
    
    parameters:
        lst: list.
        List to be segmented.
        
        size: int.
        Size of the chunks.
    """
    
    for i in range(0, len(lst), size):
        
        yield lst[i:i+size]
        

In [69]:
def clean_stats(stats):
    """
    Clean the statistics scraped from the webpage. Remove ',' and '\n'.
    
    parameters:
        stats: list.
        A list of statistics.
        
    returns:
        cleaned_stats: list.
        A cleaned list of statistics.
    """
    
    cleaned_stats = []
    
    for stat in stats:
        
        stat = re.sub(',','',stat)
        stat = stat.rstrip()
        cleaned_stats.append(stat)
        
    cleaned_stats.pop()
    
    return cleaned_stats

In [70]:
def table_statistics(table_data):
    """
    Scrape overall statistics on total cases, total deaths and total recoveries from 
    the table contents on the webpage, for each country.
    
    parameters:
        table_data: str.
        Scraped table data in string format.
        
    returns:
        stats: list.
        A list of lists giving country wise statistics.
    """
    
    stats = []
    row_data = table_data.find_all('td')
    
    for data in row_data:
        
        if data.find('sup'):
            
            continue
            
        if data.find('span'):
            
            stats.append('null')
            continue
        
        stats.append(data.contents[0])
        
    stats = clean_stats(stats)    
    stats = list(chunks(stats,3))
        
    return stats

In [71]:
def scrape_overall_data():
    """
    Scrape overall statistics country wise from the Wikipedia page on COVID-19 pandemic into a DataFrame.
    Writes the DataFrame to a csv file.
    
    returns: bool.
    
    """
    
    url = 'https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory'
    table_data = table_contents(url)
    
    countries = country_names(table_data)
    statistics = table_statistics(table_data)
    
    statistics_dict = {}
    
    for country, statistic in zip(countries, statistics):
        
        statistics_dict[country] = statistic
        
    dataframe = pd.DataFrame.from_dict(statistics_dict, orient= 'index', columns= [
        'total_cases',
        'total_deaths',
        'total_recoveries'
    ]).reset_index()
    
    dataframe.rename(columns= {'index':'country'}, inplace= True)
    dataframe.to_csv('./Data/covid19_overall_stat.csv', index= False)
    
    print("Successfully scraped table")
    
    return True

In [288]:
scrape_overall_data()

Successfully scraped table


True

In [289]:
pd.read_csv('./Data/covid19_overall_stat.csv')

,country,total_cases,total_deaths,total_recoveries
0,United States,1786171,104235,385125.0
1,Brazil,469510,28015,189476.0
2,Russia,396575,4555,167469.0
3,United Kingdom,272826,38376,NaN
4,Spain,238564,27125,150376.0
...,...,...,...,...
223,Saba,3,0,3.0
224,Bonaire,2,0,2.0
225,Lesotho,2,0,1.0
226,Sint Eustatius,2,0,2.0


In [100]:
def date_check(page_content):
    """
    Checks the date for the most recent update for statistics on the webpage.
    
    parameters:
        page_content: str.
        HTML parsed contents of the page.
        
    returns:
        result: bool.
        Boolean.
    """
    
    page_date = page_content.find('div', class_= 'news_date').h4.contents[0]
    page_date = re.sub('\(.*\)','',page_date)
    date = datetime.datetime.strptime(page_date + "2020", '%b %d %Y').date()
    
    if date == datetime.datetime.today().date():
        
        return False
    
    return True

In [82]:
def updated_stats(url):
    """
    Gathers updated data on the number of cases and deaths in a day.
    
    paramters:
        url: str.
        URL from where the updates are scraped.
        
    returns:
        result: tuple.
        A tuple with total cases and total deaths of the day.
    """
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    if date_check(soup):
        
        return ()
    
    updated_list = soup.find('li', class_= 'news_li')
    updates = updated_list.find_all('strong')
    
    if 'new' in updates[0].contents[0]:
        
        daily_cases = updates[0].contents[0]
        daily_cases = re.sub('[, new cases]','',daily_cases)
        daily_cases = int(daily_cases)
        
    else:
        
        daily_cases = np.NaN
        
    if 'new' in updates[1].contents[0]:
        
        daily_deaths = updates[1].contents[0]
        daily_deaths = re.sub('[, new deaths]','',daily_deaths)
        daily_deaths = int(daily_deaths)
    
    else:
        
        daily_deaths = np.NaN
    
    
    result = (daily_cases, daily_deaths)
    
    return result

In [75]:
country_mapping = {
    'us':'United States',
    'brazil':'Brazil',
    'russia':'Russia',
    'spain':'Spain',
    'italy':'Italy',
    'france':'France',
    'germany':'Germany',
    'turkey':'Turkey',
    'india':'India',
    'iran':'Iran',
    'peru':'Peru',
    'canada':'Canada',
    'chile':'Chile',
    'china':'China',
    'mexico':'Mexico',
    'saudi-arabia':'Saudi Arabia',
    'pakistan':'Pakistan',
    'belgium':'Belgium',
    'qatar':'Qatar',
    'bangladesh':'Bangladesh',
    'belarus':'Belarus',
    'ecuador':'Ecuador',
    'sweden':'Sweden'
}

In [110]:
def daily_updates():
    """
    Scrape daily updates on covid-19 statistics and update data files.
    
    returns: bool.
    Boolean.
    """
    
    for country in country_mapping.keys():
        
        url = "https://www.worldometers.info/coronavirus/country/"+country+"/"
        updates = updated_stats(url)
        
        country_df = pd.read_csv('./Data/covid19_'+country+'_stats.csv')
        overall_data = pd.read_csv('./Data/covid19_overall_stat.csv')
        
        overall_county_data = overall_data[overall_data.country == country_mapping[country]]
        
        last_update = country_df.iloc[-1]
        new_update = last_update
        
        new_update.date = datetime.datetime.today().date()
        
        if updates:
            
            new_update.total_cases = last_update.total_cases + updates[0]
            new_update.daily_cases = updates[0]
            
            if not np.isnan(updates[1]):
                
                new_update.total_deaths = last_update.total_deaths + updates[1]
                new_update.daily_deaths = updates[1]
                
            else:
                
                new_update.daily_deaths = np.NaN
                
            new_update.active_cases = new_update.total_cases - (
                overall_county_data.total_recoveries.iloc[0] + overall_county_data.total_deaths.iloc[0]
            )
        
            country_df = country_df.append(new_update).reset_index(drop= True)
            country_df.to_csv('./Data/covid19_'+country+'_stats.csv', index=False)
            print('Successfully updated: ',country)
            
        else:
            
            print("No updates: ", country)
        
    return True

In [296]:
daily_updates()

/home/hp/anaconda3/envs/r_env/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


No updates:  us
Successfully updated:  brazil
No updates:  russia
No updates:  spain
No updates:  italy
No updates:  france
No updates:  germany
No updates:  turkey
No updates:  india
No updates:  iran
No updates:  peru
No updates:  canada
No updates:  chile
Successfully updated:  china
Successfully updated:  mexico
No updates:  saudi-arabia
Successfully updated:  pakistan
No updates:  belgium
No updates:  qatar
No updates:  bangladesh
No updates:  belarus
No updates:  ecuador
No updates:  sweden


True

In [291]:
pd.read_csv('./Data/covid19_india_stats.csv')

,date,total_cases,daily_cases,active_cases,total_deaths,daily_deaths
0,2020-02-15,3,NaN,0.0,0,NaN
1,2020-02-16,3,0.0,0.0,0,NaN
2,2020-02-17,3,0.0,0.0,0,NaN
3,2020-02-18,3,0.0,0.0,0,NaN
4,2020-02-19,3,0.0,0.0,0,NaN
...,...,...,...,...,...,...
101,2020-05-26,150793,5843.0,82172.0,4344,172.0
102,2020-05-27,158086,7293.0,85803.0,4534,190.0
103,2020-05-28,165386,7300.0,89755.0,4711,177.0
104,2020-05-29,173491,8105.0,85884.0,4980,269.0


In [116]:
from psycopg2 import connect


In [131]:
conn = psycopg2.connect("dbname=covid19_stats user=hp password=redblue22 host=127.0.0.1")
conn.autocommit = True
cursor = conn.cursor()

In [245]:
query = """SELECT date FROM india_stats ORDER BY date DESC LIMIT 1"""

cursor.execute(query)
cursor.fetchone()[0] == datetime.datetime.today().date()

True

In [146]:
row

date            2020-02-16
total_cases              0
daily_cases              0
active_cases             0
total_deaths             0
daily_deaths             0
Name: 1, dtype: object

In [148]:
row = df.iloc[1].fillna(0)
query = """
            INSERT INTO """+'india'+"""_stats
            VALUES (
            '"""+row.date+"""',
            """+str(row.total_cases)+""",
            """+str(row.daily_cases)+""",
            """+str(row.active_cases)+""",
            """+str(row.total_deaths)+""",
            """+str(row.daily_deaths)+"""
            );
            """

cursor.execute(query)

In [115]:
conn

<connection object at 0x7fec19556b90; dsn: 'user=hp password=xxx dbname=covid19_stats host=127.0.0.1', closed: 0>

In [313]:
def connect_database(dbname, user='hp', password='test1234', host='127.0.0.1', autocommit=False):
    """
    Connect to the PostgreSQL database based on the parameters.
    
    parameters:
        dbname: str.
        Name of the database.
        
        user: str.
        Name of the user. Default hp.
        
        password: str.
        Password for the user to connect.
        
        host: str.
        The IP address of the hosted database. Default 127.0.0.1 (localhost).
        
        autocommit: Boolean.
        Set autocommit to True or False. Default False
        
    returns:
        conn: connection object.
        A connection object to the database.
    """
    
    conn = connect("dbname="+dbname+" user="+user+" password="+password+" host="+host)
    conn.autocommit = True
    
    return conn

In [124]:
connect_database('covid19_stats',password='redblue22',autocommit= True)c

<connection object at 0x7fec19f6fcd0; dsn: 'user=hp password=xxx dbname=covid19_stats host=127.0.0.1', closed: 0>

In [162]:
def create_country_relations(conn):
    """
    Create relations for every country in the database, for covid-19 statistics.
    
    parameters:
        conn: connection object.
        A connection object to the database.
        
    returns: bool.
    Boolean.
    """
    
    cursor = conn.cursor()
    
    for country in countries:
        
        data = pd.read_csv('./Data/covid19_'+country+'_stats.csv')
        country = re.sub('-','',country)
        
        query = "DROP TABLE IF EXISTS "+country+"_stats;"
        cursor.execute(query)
        
        query = """
        CREATE TABLE """+country+"""_stats (
            date DATE PRIMARY KEY, 
            total_cases INT,
            daily_cases INT,
            active_cases INT,
            total_deaths INT,
            daily_deaths INT
        );
        """
        
        cursor.execute(query)
        
        for ind,row in data.iterrows():
            
            row = row.fillna(0)
            
            query = """
            INSERT INTO """+country+"""_stats
            VALUES (
            '"""+row.date+"""',
            """+str(row.total_cases)+""",
            """+str(row.daily_cases)+""",
            """+str(row.active_cases)+""",
            """+str(row.total_deaths)+""",
            """+str(row.daily_deaths)+"""
            );"""
            
            cursor.execute(query)
            
        print('Table successfully created: ',country+'_stats')
        
    return True

In [292]:
create_country_relations(connect_database('covid19_stats',password='redblue22',autocommit= True))

Table successfully created:  us_stats
Table successfully created:  brazil_stats
Table successfully created:  russia_stats
Table successfully created:  spain_stats
Table successfully created:  italy_stats
Table successfully created:  france_stats
Table successfully created:  germany_stats
Table successfully created:  turkey_stats
Table successfully created:  india_stats
Table successfully created:  iran_stats
Table successfully created:  peru_stats
Table successfully created:  canada_stats
Table successfully created:  chile_stats
Table successfully created:  china_stats
Table successfully created:  mexico_stats
Table successfully created:  saudiarabia_stats
Table successfully created:  pakistan_stats
Table successfully created:  belgium_stats
Table successfully created:  qatar_stats
Table successfully created:  bangladesh_stats
Table successfully created:  belarus_stats
Table successfully created:  ecuador_stats
Table successfully created:  sweden_stats


True

In [205]:
def create_overall_relation(conn):
    """
    Create the relation containing overall statistics for all the countries.
    
    parameters:
        conn: connection object.
        A connection object to the database.
        
    returns: bool.
    Boolean.
    """
    
    cursor = conn.cursor()
    
    data = pd.read_csv('./Data/covid19_overall_stat.csv')
    
    query = "DROP TABLE IF EXISTS overall_stats;"
    cursor.execute(query)
        
    query = """
        CREATE TABLE overall_stats (
            country VARCHAR(25), 
            total_cases BIGINT,
            total_deaths BIGINT,
            total_recoveries BIGINT
        );
        """
    cursor.execute(query)
    
    for ind,row in data.iterrows():
        
        row = row.fillna(0)
        row.country = row.country.replace(' ','')
        
        query = """
            INSERT INTO overall_stats VALUES (
            '"""+row.country+"""',
            """+str(row.total_cases)+""",
            """+str(row.total_deaths)+""",
            """+str(row.total_recoveries)+"""
            );"""
        
        cursor.execute(query)
        
    print("Table successfully created")
    
    return True

In [293]:
create_overall_relation(connect_database('covid19_stats',password='redblue22',autocommit= True))

Table successfully created


True

In [304]:
def update_database(conn):
    """
    Updates the database with the daily updates on statistics for countries.
    
    parameters:
        conn: connection object.
        A connection object to the database.
        
    returns: bool.
    Boolean.
    """
    
    cursor = conn.cursor()
    
    for country in countries:
        
        query = "SELECT date FROM "+country.replace('-','')+"_stats ORDER BY date DESC LIMIT 1;"
        cursor.execute(query)
        last_date = cursor.fetchone()[0]
        
        if (pd.to_datetime(last_date).date() != datetime.datetime.today().date()):
            
            data = pd.read_csv('./Data/covid19_'+country+'_stats.csv')
            update = data.iloc[-1].fillna(0)
            
            if pd.to_datetime(update.date).date() != datetime.datetime.today().date():
                
                continue
            
            query = """
            INSERT INTO """+country.replace('-','')+"""_stats
            VALUES (
            '"""+update.date+"""',
            """+str(update.total_cases)+""",
            """+str(update.daily_cases)+""",
            """+str(update.active_cases)+""",
            """+str(update.total_deaths)+""",
            """+str(update.daily_deaths)+"""
            );
            """
            
            cursor.execute(query)
            
            print("Table",country+"_stats updated")
            
    
    return True
    

In [312]:
update_database(connect_database('covid19_stats',password='test1234',autocommit= True))

True

In [309]:
connect_database('covid19_stats',password='test1234',autocommit=True)

<connection object at 0x7fec11123050; dsn: 'user=hp password=xxx dbname=covid19_stats host=127.0.0.1', closed: 0>